In [19]:
import numpy as np
import random

In [20]:
!pip install pyspark

Defaulting to user installation because normal site-packages is not writeable


In [21]:
from pyspark.sql import SparkSession
spark = SparkSession\
    .builder\
    .appName("recommender")\
    .getOrCreate()
sc = spark.sparkContext

In [22]:
data = sc.textFile("ml-100k/u.data")
header = data.first() #extract header
data = data.filter(lambda row: row != header)
M = data.map(
  lambda l: l.split('\t')
).map(
  lambda l: (int(l[0]), int(l[1]), float(l[2]))
)


In [23]:
numWorkers = sc.defaultParallelism
numWorkers

4

In [24]:
M.max(lambda x: x[0])

(943, 58, 4.0)

In [25]:
def assignBlockIndex (index, numData, numWorkers):
    blockSize = numData/numWorkers
    if(numData % numWorkers != 0): blockSize = blockSize + 1
    return int(np.floor(index/np.ceil(blockSize)))+1


In [26]:
numFactors = 10
numRows = M.max(lambda x : x[0])[0] + 1
numCols = M.max(lambda x : x[1])[1] + 1
avgRating = M.map(lambda x: x[2]).mean()

#compute the scaling factor that the randomly initialized W and H matrices need to be scaled by so that dot(W_0,H_0) results in values that are similar to the average ratings
scaleRating = np.sqrt(avgRating / numFactors)

tau = 100


In [27]:
W = M.map(lambda x: tuple([int(x[0]),1])).reduceByKey(lambda x,y : x+y).map(lambda x: tuple([x[0], tuple([x[1], np.random.rand(1,numFactors).astype('float16')])])).persist()
H = M.map(lambda x: tuple([int(x[1]),1])).reduceByKey(lambda x,y : x+y).map(lambda x: tuple([x[0], tuple([x[1], np.random.rand(1,numFactors).astype('float16')])])).persist()

Iterater turn

In [28]:
it = 1
beta = 0.1
mse = sc.accumulator(0.0)
nUpdates = sc.accumulator(0.0)
#broadcast the stepsize
#stepSize = sc.broadcast(np.power(tau + it, -beta))
stepSize = sc.broadcast(0.5)
#generate random strata
lam = sc.broadcast(0.1)

perms = np.random.permutation(numWorkers)+1
perms

array([2, 4, 3, 1])

In [29]:
Mblocked = M.keyBy(lambda x: assignBlockIndex(x[0], numRows, numWorkers)).partitionBy(numWorkers)

In [30]:
def SGD(keyed_iterable, stepSize, numFactors,lam, mse, nUpdates, scaleRating):
    iterlist = (next(keyed_iterable))
    key = iterlist[0]
    Miter = iterlist[1][0]
    Hiter = iterlist[1][1]
    Witer = iterlist[1][2]
    
    Wdict = {}
    Hdict = {}
    
    Wout = {}
    Hout = {}
    
    #iterate through H and W and create dictionary of elements
    for h in Hiter:
        Hdict[h[0]] = h[1]
    
    for w in Witer:
        Wdict[w[0]] = w[1]
    #iterate through entries of M and compute L2-loss
    counter = 1
    for m in Miter:
        (i,j,rat) = m
        if i not in Wdict:
            Wdict[i] = tuple([i,np.random.uniform(0,1,numFactors).astype('float32')])
        if j not in Hdict:
            Hdict[j] = tuple([j,np.random.uniform(0,1,numFactors).astype('float32')])

        (Nw, Wprev) = Wdict[i]
        (Nh, Hprev) = Hdict[j]
        error = (rat - np.dot(Wprev, Hprev.T))
        mse += pow((error[0][0]), 2)
        
        Wnew = Wprev - stepSize.value*(-2*error*Hprev+ (2.0*lam.value)*Wprev)
        Hnew = Hprev - stepSize.value*(-2*error*Wprev + (2.0*lam.value)*Hprev)
        nUpdates += 1
        Wdict[i] = tuple([Nw, Wnew])
        Hdict[j] = tuple([Nh, Hnew])
        
    return (tuple(['W',Wdict.items()]), tuple(['H',Hdict.items()]))

In [31]:
#mse = sc.accumulator(0.0)
#nUpdates = sc.accumulator(0)
stepSize = 0.01
max_iter = 100
stepSize = sc.broadcast(stepSize)
for it in range(max_iter):
    mse = sc.accumulator(0.0)
    nUpdates = sc.accumulator(0)

    beta = 0.1
    #broadcast the stepsize
    #stepSize = sc.broadcast(1/np.power(tau + it, beta))
    stepSize = sc.broadcast(stepSize.value * 0.9)
    #generate random strata
    lam = sc.broadcast(0.01)

    perms = np.random.permutation(numWorkers)+1
    perms_dict = {i: val for i, val in enumerate(perms)}
    rev_perms=list(i+1 for i in (dict(sorted(perms_dict.items(), key=lambda item: item[1]))).keys())
    Mfilt = Mblocked.filter(lambda x: perms[x[0]-1]==assignBlockIndex(x[1][1],numCols,numWorkers)).persist()
    Hblocked = H.keyBy(lambda x: rev_perms[assignBlockIndex(x[0], numCols, numWorkers)-1])
    Wblocked = W.keyBy(lambda x: assignBlockIndex(x[0], numRows, numWorkers))
    groupRDD = Mfilt.groupWith(Hblocked, Wblocked).partitionBy(numWorkers)
    WH = groupRDD.mapPartitions(lambda x: SGD(x, stepSize, numFactors,lam, mse, nUpdates,scaleRating))
    W.unpersist()
    H.unpersist()
    W = WH.filter(lambda x: x[0]=='W').flatMap(lambda x: x[1]).persist()
    H = WH.filter(lambda x: x[0]=='H').flatMap(lambda x: x[1]).persist()
    Wvec = W.collect()
    Hvec = H.collect()
    print("MSE/update for {}-th iteration is: {}/{} ".format(it, mse.value, nUpdates.value))
    print("RMSE: {}".format((mse.value/nUpdates.value)))


MSE/update for 0-th iteration is: 70622.98502588272/51806 
RMSE: 1.3632201873505525


MSE/update for 1-th iteration is: 53182.59436607361/52344 
RMSE: 1.0160208307747518


MSE/update for 2-th iteration is: 49432.68571329117/51240 
RMSE: 0.9647284487371423


MSE/update for 3-th iteration is: 47256.09651851654/48286 
RMSE: 0.9786707641659392


MSE/update for 4-th iteration is: 49056.025265693665/53266 
RMSE: 0.9209631897588267


...

MSE/update for 95-th iteration is: 39288.755153656006/48208 
RMSE: 0.8149841344518753


MSE/update for 96-th iteration is: 40268.2315530777/47206 
RMSE: 0.8530320627267233


MSE/update for 97-th iteration is: 38422.84214544296/48236 
RMSE: 0.79655946068171


MSE/update for 98-th iteration is: 38482.13399839401/47336 
RMSE: 0.8129570305558985


MSE/update for 99-th iteration is: 39440.781863212585/51240 
RMSE: 0.7697264219986844


In [32]:
M.filter(lambda x: x[0]==15).collect()

[(15, 405, 2.0),
 (15, 749, 1.0),
 (15, 25, 3.0),
 (15, 331, 3.0),
 (15, 222, 3.0),
 (15, 473, 1.0),
 (15, 678, 1.0),
 (15, 932, 1.0),
 (15, 127, 2.0),
 (15, 685, 4.0),
 (15, 20, 3.0),
 (15, 301, 4.0),
 (15, 278, 1.0),
 (15, 620, 4.0),
 (15, 742, 2.0),
 (15, 137, 4.0),
 (15, 696, 2.0),
 (15, 924, 3.0),
 (15, 289, 3.0),
 (15, 508, 2.0),
 (15, 754, 5.0),
 (15, 18, 1.0),
 (15, 286, 2.0),
 (15, 148, 3.0),
 (15, 864, 4.0),
 (15, 244, 2.0),
 (15, 274, 4.0),
 (15, 9, 4.0),
 (15, 307, 1.0),
 (15, 458, 5.0),
 (15, 476, 4.0),
 (15, 471, 4.0),
 (15, 7, 1.0),
 (15, 937, 4.0),
 (15, 929, 1.0),
 (15, 889, 3.0),
 (15, 591, 2.0),
 (15, 1, 1.0),
 (15, 933, 1.0),
 (15, 459, 5.0),
 (15, 411, 2.0),
 (15, 744, 4.0),
 (15, 815, 1.0),
 (15, 300, 4.0),
 (15, 926, 1.0),
 (15, 409, 3.0),
 (15, 308, 5.0),
 (15, 303, 3.0),
 (15, 251, 2.0),
 (15, 936, 5.0),
 (15, 455, 1.0),
 (15, 306, 5.0),
 (15, 928, 1.0),
 (15, 14, 4.0),
 (15, 269, 5.0),
 (15, 13, 1.0),
 (15, 252, 2.0),
 (15, 927, 4.0),
 (15, 255, 5.0),
 (15, 50

In [33]:
W.filter(lambda x: x[0]==186).map(lambda x: x[1][1]).collect()[0]

array([[0.842 , 0.785 , 0.225 , 0.2095, 0.489 , 0.579 , 0.983 , 0.8164,
        0.405 , 0.738 ]], dtype=float16)

In [34]:
w_i = np.array(W.filter(lambda x: x[0]==15).map(lambda x: x[1][1]).collect()[0])
w_j = np.array(W.filter(lambda x: x[0]==222).map(lambda x: x[1][1]).collect()[0])
np.dot(w_i, w_j.T)

array([[2.908]], dtype=float16)